In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.optimize as opt
import codecs as cod
import scipy.special as sf
import scipy.stats as stat
import matplotlib.colors as mcolors
import scipy.interpolate as interp
from scipy import odr

### Functions and Definitions

In [ ]:
def dimension_check(a, b, c):
    """Dimension Check:
    Returns true if the mechanism is mathematically tenable; returns false otherwise.
    
    a = crank length (m)
    b = connecting shaft length (m)
    c = displacement between rotational and reciprocating axes (m)"""
    
    if ((a + c) >= b) or (a < 0) or (b < 0) or (c < 0):
        return False
    else:
        return True
    
def crank_ang(t, rpm):
    """Crank Angle:
    Returns the angle of the crank (in radians) as a function of rotations per minute and time.
    
    t = time (s)
    rpm = rotations per minute (rev/min)"""
    
    return rpm * 2 * np.pi * t / 60

def rod_ang_pol(t, rpm, a, b, c):
    """Rod Angle Polar:
    Returns the polar angle of the connecting shaft (angle from z-axis).
    
    t = time (s)
    rpm = rotations per minute (rev/min)
    a = crank length (m)
    b = connecting shaft length (m)
    c = displacement between rotational and reciprocating axes (m)"""
    
    if not dimension_check(a, b, c):
        return 0
    else:
        c_ang = crank_ang(t, rpm)
        return np.arccos(-(a * np.sin(c_ang) + c) / b)

def rod_ang_az(t, rpm, a, b, c):
    """Rod Angle Azimuthal:
    Returns the azimuthal angle of the connecting shaft (angle from x-axis)
    
    t = time (s)
    rpm = rotations per minute (rev/min)
    a = crank length (m)
    b = connecting shaft length (m)
    c = displacement between rotational and reciprocating axes (m)"""
    
    if not dimension_check(a, b, c):
        return 0
    else:
        c_ang = crank_ang(t, rpm)
        r_ang_p = rod_ang_pol(t, rpm, a, b, c)
        return np.arccos(-a * np.cos(c_ang) / (b * np.sin(r_ang_p)))

def slide_pos(t, rpm, a, b, c):
    """Slider Position:
    Returns the horizontal distance from the crank to the slider.
    
    t = time (s)
    rpm = rotations per minute (rev/min)
    a = crank length (m)
    b = connecting shaft length (m)
    c = displacement between rotational and reciprocating axes (m)"""
    
    if not dimension_check(a, b, c):
        return 0
    else:
        c_ang = crank_ang(t, rpm)
        r_ang_p = rod_ang_pol(t, rpm, a, b, c)
        r_ang_a = rod_ang_az(t, rpm, a, b, c)
        return b * np.sin(r_ang_a) * np.sin(r_ang_p)

def slide_vel(t, rpm, a, b, c):
    """"""
    
    if not dimension_check(a, b, c):
        return 0
    else:
        theta = crank_ang(t, rpm)
        num = - (a * c * np.cos(theta))
        denom = b * np.sqrt((b**2 - (c + a * np.sin(theta))**2) / b**2) * np.sqrt(1 + ((a**2 * np.cos(theta)**2) / ((-b +c + a * np.sin(theta)) * (b + c + a * np.sin(theta)))))
        return (num / denom) * 2 * np.pi * rpm / 60

#def max_vel(rpm, a, b, c):
#    """"""
#    
#    if not dimension_check(a, b, c):
#        return 0
#    else:
#        return np.abs(slide_vel(0, rpm, a, b, c))

def max_vel(rpm, a, b, c):
    """"""
    
    if not dimension_check(a, b, c):
        return 0
    else:
        return (a * c) / (b * np.sqrt(1 - (c / b)**2) * np.sqrt(1 - a**2 / (b**2 - c**2))) * 2 * np.pi * rpm / 60

def stroke_len(a, b, c):
    """Stroke Length:
    Returns the length of the stroke; it is the maximum slider position minus the minimum slider position.
    
    a = crank length (m)
    b = connecting shaft length (m)
    c = displacement between rotational and reciprocating axes (m)"""
    
    if not dimension_check(a, b, c):
        return 0
    else:
        l_min = np.sqrt(b**2 - (a + c)**2)
        l_max = np.sqrt(b**2 - (a - c)**2)
        return l_max - l_min

def b_s_ang(a, b, c):
    """Ball Swivel Angle:
    Returns the ball swivel angle (in degrees): twice the angle of declination of the rod when the crank is in the peak position (straight up).
    
    a = crank length (m)
    b = connecting shaft length (m)
    c = displacement between rotational and reciprocating axes (m)"""
    
    if not dimension_check(a, b, c):
        return 0
    else:
        return np.rad2deg(2 * np.arcsin((a + c) / b))

def b_s_ang_test(b_s_al, a, b, c):
    """Ball Swivel Angle Test:
    Returns true if the calculated ball swivel angle is less than or equal to the allowable ball swivel angle; returns false otherwise.
    
    b_s_al = allowable ball swivel angle (degrees)
    a = crank length (m)
    b = connecting shaft length (m)
    c = displacement between rotational and reciprocating axes (m)"""
    
    if not dimension_check(a, b, c):
        return False
    else:
        ang = b_s_ang(a, b, c)
        if ang > b_s_al:
            return False
        else:
            return True

def f_top_pmass(rpm, a, b, c):
    """Top Force per unit Mass:
    Returns the force per unit mass (N/kg) going through the connecting rod when the crank is in its crest position."""
    
    if not dimension_check(a, b, c):
        return 0
    else:
        omega = 2 * np.pi *rpm / 60
        return - (a * b * c) / ((a - b + c) * (a + b + c)) * omega**2

def f_bot_pmass(rpm, a, b, c):
    """Bottom Force per unit Mass:
    Returns the force per unit mass (N/kg) going through the connecting rod when the crank is in its trough position."""
    
    if not dimension_check(a, b, c):
        return 0
    else:
        omega = 2 * np.pi * rpm / 60
        return (a * b * c) / (a**2 - b**2 - 2*a*c + c**2) * omega**2

def m2in(x):
    return 39.3701 * x

def in2m(x):
    return 0.0254 * x

def stroke_len_max(ang, a):
    """"""
    
    b = a * 2 / np.sin(np.deg2rad(ang) / 2)
    return b - np.sqrt(b**2 - 4 * a**2)

In [ ]:
rpm_test = 2500
#rpm_test is rotations per minute; this is the rate of the hand drill.
a_test = in2m(1.25)
#a_test is the length of the crank; it is in meters.
b_test = in2m(9)
#b_test is the length of the connecting rod; it is in meters.
c_test = in2m(1.25)
#c_test is the distance between the axis of rotation and axis of reciprocation; it is in meters.
b_s_al = 45
#b_s_al is the allowable ball socket angle; it is in degrees.
t_start = 0
t_end = 60 / rpm_test
#t_start and t_end are the start and end times (respectively); they are in seconds.

round_count = 1
#round_count is the number of decimal places kept in numbers appearing in figures.

### Test

In [ ]:
t = np.linspace(t_start, t_end, 10000)
#t is the time (in s) over which we investigate.

s_pos = slide_pos(t, rpm_test, a_test, b_test, c_test)
s_vel = slide_vel(t, rpm_test, a_test, b_test, c_test)
#s_pos and s_vel are the slider position and velocity (in m/s), respectively.

s_l = stroke_len(a_test, b_test, c_test)
max_v = max_vel(rpm_test, a_test, b_test, c_test)
#s_l is the stroke length (in m); max_vel is the maximum velocity (in m/s).

c_ang = np.rad2deg(crank_ang(t, rpm_test))
#c_ang is the crank angle in degrees.

f_t_pm = f_top_pmass(rpm_test, a_test, b_test, c_test)
f_b_pm = f_bot_pmass(rpm_test, a_test, b_test, c_test)

print(f"""Does this mechanism pass the angle requirement?
{b_s_ang_test(b_s_al, a_test , b_test , c_test)}""")

In [ ]:
c_ang_pos_max = np.round(c_ang[np.where(s_pos == max(s_pos))[0][0]], round_count)
c_ang_pos_min = np.round(c_ang[np.where(s_pos == min(s_pos))[0][0]], round_count)
s_pos_max = max(s_pos)
s_pos_min = min(s_pos)
pic_pos_h = s_pos_max - s_pos_min

c_ang_vel_max = np.round(c_ang[np.where(s_vel == max(s_vel))[0][0]], round_count)
c_ang_vel_min = np.round(c_ang[np.where(s_vel == min(s_vel))[0][0]], round_count)
s_vel_max = max(s_vel)
s_vel_min = min(s_vel)
pic_vel_h = s_vel_max - s_vel_min

In [ ]:
fig = plt.figure(figsize=(6,8))

ax1 = fig.add_subplot(211)
ax1.set_title("Slider Position over Time and Angle")
ax1.set_xlabel("Time (s)")
ax1.set_ylabel("Slider Position (m)")
ax1.plot(t, s_pos, c = 'r', label = "wrt time")
ax1.set_xlim(t[0], t[-1])
ax1.legend(loc='best')

ax2 = fig.add_subplot(212)
ax2.set_xlabel("Angle (degree)")
ax2.set_ylabel("Slider Position (m)")
ax2.plot(c_ang, s_pos, c = 'r', ls = '--', label = "wrt angle")
ax2.set_xlim(c_ang[0], c_ang[-1])
ax2.axvline(x = c_ang_pos_max, ymin=0, ymax=1, c='k', ls='--')
ax2.axvline(x = c_ang_pos_min, ymin=0, ymax=1, c='k', ls='--')
ax2.legend(loc = 'best')
ax2.text(c_ang_pos_min, s_pos_max - pic_pos_h * 0.1, f"Max angle = {c_ang_pos_max}")
ax2.text(c_ang_pos_min, s_pos_max - pic_pos_h * 0.2, f"Min angle = {c_ang_pos_min}")
#ax.axvline(x=270,ymin=0.,ymax=10., c='k', ls='--')

print(f"""Value:
Stroke length: {s_l} m
Stroke length: {m2in(s_l)} in""")
plt.savefig("Slider_Pos.png")

In [ ]:
fig = plt.figure(figsize=(6,8))

ax3 = fig.add_subplot(211)
ax3.set_title("Slider Velocity over Time and Angle")
ax3.set_xlabel("Time (s)")
ax3.set_ylabel("Slider Velocity (m/s)")
ax3.plot(t, s_vel, c = 'b', label = "wrt time")
ax3.set_xlim(t[0], t[-1])
ax3.legend(loc='best')

ax4 = fig.add_subplot(212)
ax4.set_xlabel("Angle (degree)")
ax4.set_ylabel("Slider Velocity (m/s)")
ax4.plot(c_ang, s_vel, c = 'b', ls = '--', label = "wrt angle")
ax4.set_xlim(c_ang[0], c_ang[-1])
#ax4.axvline(x = c_ang_vel_max, ymin=0, ymax=1, c='k', ls='--')
#ax4.axvline(x = c_ang_vel_min, ymin=0, ymax=1, c='k', ls='--')
ax4.legend(loc = 'best')
#ax4.text(c_ang_vel_max, s_vel_max - pic_vel_h * 0.1, f"Max angle = {c_ang_vel_max}")
#ax4.text(c_ang_vel_max, s_vel_max - pic_vel_h * 0.2, f"Min angle = {c_ang_vel_min}")

print(f"""Value:
Slider velocity: {s_vel_max} m/s
Slider velocity: {m2in(s_vel_max)} in/s""")
plt.savefig("Slider_Vel.png")

#### Force Analytic

In [ ]:
print(f"""Connecting Shaft Force (Analytic):
Top F/m = {f_t_pm}
Bottom F/m = {f_b_pm}""")
omega = 2 * np.pi * rpm_test / 60
print(-a_test**2 / b_test * omega**2)

#### Force Numerical

In [ ]:
accel = interp.InterpolatedUnivariateSpline(t, s_vel).derivative()
accel_max = max(accel(t))
accel_min = min(accel(t))
phi_t = np.arcsin((a_test + c_test) / b_test)
phi_b = np.arcsin((a_test - c_test) / b_test)
#print(accel_max/np.cos(phi_t))
#print(accel_min/np.cos(phi_b))
print(f"""Connecting Shaft Force (Numeric):
Top F/m = {accel_max/np.cos(phi_t)}
Bottom F/m = {accel_min/np.cos(phi_b)}""")
#This corroborates the analytic solutions.

#### Axis Displacement

In [ ]:
a_test = in2m(1.25)
b_test = in2m(9)

c_range = np.linspace(0, in2m(10), 1000)
s_l_range = np.zeros(len(c_range))
for i in range(0, len(c_range)):
    s_l_range[i] = stroke_len(a_test, b_test, c_range[i])
plt.plot(m2in(c_range), m2in(s_l_range))

### Dimensional Analysis

To optimize the stroke lenght, we take that a=c.

#### Define Terms

In [ ]:
rpm = 2500
#rpm_test is rotations per minute; this is the rate of the hand drill.
a_lim = in2m(np.array([0.25,1.75]))
#a is the length of the crank; it is in meters.
b_lim = in2m(np.array([1.0,12.0]))
#b_lim is an array with the lower and upper limit of the the connecting shaft; it is in meters.
b_s_al = 45
#b_s_al is the allowable ball socket angle; it is in degrees.
t_start = 0
t_end = 60 / rpm
#t_start and t_end are the start and end times (respectively); they are in seconds.

round_count = 1
#round_count is the number of decimal places kept in numbers appearing in figures.

In [ ]:
#a = np.linspace(a_lim[0], a_lim[1], 1000)
#b = np.linspace(b_lim[0], b_lim[1], 1000)
#c = a
#data = np.zeros((3, len(a)))
#for i in range(0, len(a)):
#    data[0] = a
#    s_l = np.zeros(len(b))
#    for j in range (0, len(b)):
#        s_l[j] = stroke_len(a[i], b[j], c[i])
#    s_l_max = max(s_l)
#    b_max = b[np.where(s_l == s_l_max)[0][0]]
#    data[1, i] = s_l_max
#    data[2, i] = b_max

data[0] is the length of the crank arm and axes displacement.
<br>data[1] is the maximum stroke length.
<br>data[2] is the lenght of the connecting shaft at which this maximum occurs.
<br>The problem is that the maximum stroke length occurs when the connecting shaft is closest to the lever arm length. But such a system is physically infeasible. We need to include the limiting ball socket angle.

In [ ]:
a = in2m(np.array([0.25,0.375,0.5,0.625,0.75,0.875,1.0,1.125,1.25,1.375,1.5,1.625,1.75]))
b = np.linspace(b_lim[0], b_lim[1], 1000)
c = a
data = np.zeros((3, len(a)))

for i in range(0, len(a)):
    data[0] = m2in(a)
    test = np.zeros(len(b), dtype = bool)
    s_l = np.zeros(len(b))
    for j in range (0, len(b)):
        s_l[j] = stroke_len(a[i], b[j], c[i])
        test[j] = b_s_ang_test(b_s_al, a[i], b[j], c[i])
    s_l_max = max(s_l)
    b_max = b[np.where(s_l == s_l_max)[0][0]]
    
    s_l_n = m2in(s_l)
    b_n = m2in(b)
    s_l_max_n = m2in(s_l_max)
    w = max(b_n) - min(b_n)
    h = max(s_l_n) - min(s_l_n)
    
    allow = np.where(test == True)[0][0]
    cutoff = b_n[allow]
    s_l_cutoff = s_l_n[allow]
    s_l_max_ang = m2in(stroke_len_max(b_s_al, a[i]))
    
    data[1, i] = cutoff
    data[2, i] = s_l_max_ang
    
    fig = plt.figure(figsize=(6,8))
    ax = fig.add_subplot(111)
    ax.set_title(f"Stroke Lenght with a=c={np.round(m2in(a[i]),3)}")
    ax.set_xlabel("Connecting Rod Length (in)")
    ax.set_ylabel("Stroke Length (in)")
    ax.plot(b_n, s_l_n, c = 'b', label = f"a = c = {np.round(m2in(a[i]),3)}")
    ax.set_xlim(b_n[0], b_n[-1])
    ax.axvline(x = cutoff, c = 'k', ls = '--')
    ax.text(max(b_n) - 0.25 * w, max(s_l_n) - 0.05 * h, f"SL Max = {np.round(s_l_max_n,2)}")
    ax.text(max(b_n) - 0.25 * w, max(s_l_n) - 0.1 * h, f"SL Allow = {np.round(s_l_cutoff,2)}")
    ax.text(max(b_n) - 0.25 * w, max(s_l_n) - 0.15 * h, f"SL Allow = {np.round(s_l_max_ang,2)}")
    ax.text(max(b_n) - 0.25 * w, max(s_l_n) - 0.2 * h, f"B Allow = {np.round(cutoff,2)}")
    ax.legend(loc='best')